In [1]:
import warnings
import datetime as dt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import torchvision.transforms as T
from torchvision import datasets

warnings.filterwarnings('ignore')

### Data

In [4]:
def loadData():   
    test_T  = T.Compose([T.Resize((32,32)),
                         T.ToTensor()])
    test_data  = datasets.ImageFolder("traindata", transform=test_T)  
    
    return test_data

In [5]:
test_dl = DataLoader(loadData(), 16, shuffle=True)

### Super Class

In [6]:
class Eval(nn.Module):
    def val_step(self, batch):
        imgs, labs = batch 
        pred = self(imgs)                    # Generate predictions
        loss = F.cross_entropy(pred, labs)   # Calculate loss
        acc  = accuracy(pred, labs)          # Calculate accuracy
        return {'loss': loss.detach(), 'acc': acc}
        
    def val_epoch(self, outputs):
        batch_losses = [x['loss'] for x in outputs]
        epoch_loss   = torch.stack(batch_losses).mean()  # Combine losses
        
        batch_accs = [x['acc'] for x in outputs]
        epoch_acc  = torch.stack(batch_accs).mean()      # Combine accuracies
        
        return {'Loss': epoch_loss.item(), 'Accuracy': epoch_acc.item()}

### Model

In [8]:
class CNN_final(Eval):   
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(    
            nn.Conv2d(3, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
        
            nn.Conv2d(64, 128, 3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(128, 128, 3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Dropout(0.2),
            
            nn.Flatten(),
            nn.Linear(512,128),
            nn.ReLU(),
            nn.Dropout(0.4),
            
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64,32),
            nn.ReLU(),
            nn.Linear(32,3),
            nn.Softmax()
        )
        
    def forward(self, x):
        return self.layers(x);

### Functions

In [9]:
def accuracy(outputs, labs):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labs).item() / len(preds))

@torch.no_grad()
def evaluate(model, val_loader):
    S = dt.datetime.now()
    
    model.eval()
    outputs = [model.val_step(batch) for batch in val_loader]
    
    E = dt.datetime.now()
    
    print("\nPerformance:")
    print(model.val_epoch(outputs))
    
    print("\nExecution time:", round((E-S).total_seconds(), 4), "secs")

### Evaluation

In [11]:
CNN_final_test = CNN_final()
CNN_final_test.load_state_dict(torch.load("C:/Users/User/Documents/COMP309/model.pth", map_location = torch.device('cpu')))

<All keys matched successfully>

In [12]:
evaluate(CNN_final_test, test_dl)


Performance:
{'Loss': 0.8962659239768982, 'Accuracy': 0.6184895634651184}

Execution time: 11.3704 secs
